<a href="https://colab.research.google.com/github/jaunty0216/Final_Project_forNTNU/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E5%B0%88%E9%A1%8C%E3%80%80%E3%80%90%E4%BA%94%E6%9C%88%E3%80%91%E6%83%85%E7%B7%92%E6%8A%92%E7%99%BC%E5%B0%8F%E5%A4%A9%E5%9C%B0%E3%80%80%E6%96%87%E5%AD%97%E9%9B%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 【五月】情緒抒發小天地文字探勘

In [2]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [3]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Pdv5BZv7-JmT1kl9R4-df8tSDy3BFpP-Hc8xctrnKLM/edit?usp=sharing').sheet1

In [4]:
dicts = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Pdv5BZv7-JmT1kl9R4-df8tSDy3BFpP-Hc8xctrnKLM/edit?usp=sharing').get_worksheet(0)
dicts = dicts.get_all_records()
dicts = pd.DataFrame(dicts)

In [5]:
# 讀取所有數據
rows = gsheets.get_all_records()
import pandas as pd
df = pd.DataFrame(rows)

In [6]:
df.head()

,時間戳記,這個月有什麼煩惱呢？
0,,今天在圖書館坐了一整個下午，窗外陽光剛好灑進來，照在書頁上，整個人像是靜止在時間裡。也許這就...
1,,有些話說不出口，就變成了晚上的夢，醒來後還是帶著一點情緒，不重，但黏著心。
2,,去便利商店的路上看到一隻狗，它對我搖了搖尾巴，我也不知道為什麼就笑了。
3,,今天超級無敵煩躁，完全找不到原因，什麼都不順，看誰都不爽。我討厭這樣的自己。
4,,晚上騎腳踏車時耳機播到一首以前常聽的歌，突然就想起以前和他一起走路回家的畫面。時間過得真快，...


In [7]:
df.describe()

,時間戳記,這個月有什麼煩惱呢？
count,17,17
unique,1,17
top,,今天在圖書館坐了一整個下午，窗外陽光剛好灑進來，照在書頁上，整個人像是靜止在時間裡。也許這就...
freq,17,1


In [8]:
!pip install jieba
!pip install snownlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 52.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=97526d4ae2b6a04edb7def7010fa67444bf9ad4e241ab6d41bf2c10c03a589ef
  Stored in directory: /root/.cache/pip/wheels/4a/fc/04/d1937c02b2a445b34754da55f71612a3df648a38d711bd17eb
Successfully built snownlp


In [9]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [10]:
for _, row in dicts.iterrows():
    word = row['這個月有什麼煩惱呢？']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.343 seconds.
DEBUG:jieba:Loading model cost 1.343 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [11]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [12]:
df['Chinese_Content'] = df['這個月有什麼煩惱呢？'].apply(extract_chinese)

# 使用 Jieba 斷詞
df['Tokenized'] = df['Chinese_Content'].apply(lambda x: list(jieba.cut(x, HMM=True)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]

In [13]:
df['Chinese_Content']

,Chinese_Content
0,今天在圖書館坐了一整個下午窗外陽光剛好灑進來照在書頁上整個人像是靜止在時間裡也許這就是我一直...
1,有些話說不出口就變成了晚上的夢醒來後還是帶著一點情緒不重但黏著心
2,去便利商店的路上看到一隻狗它對我搖了搖尾巴我也不知道為什麼就笑了
3,今天超級無敵煩躁完全找不到原因什麼都不順看誰都不爽我討厭這樣的自己
4,晚上騎腳踏車時耳機播到一首以前常聽的歌突然就想起以前和他一起走路回家的畫面時間過得真快也許那...
5,終於把手邊的報告交出去了雖然不確定做得好不好但至少完成了這點就值得好好吃一頓慶祝
6,空氣裡有一種快下雨的味道這種味道總讓我想起國中的放學路
7,想說的話太多最後還是決定什麼都不說靜靜地看著聊天視窗發呆
8,今天被主管誇獎了一句沒什麼大不了但那一瞬間真的有種我是不是其實可以的的感覺
9,窗外有蟬鳴是夏天要來了吧我其實不是很喜歡夏天但這種聲音讓我覺得自己還活著


In [14]:
df['Tokenized']

,Tokenized
0,"[今天, 在, 圖書館, 坐, 了, 一整, 個, 下午, 窗外, 陽光剛, 好灑, 進來照..."
1,"[有些, 話, 說, 不, 出口, 就, 變成, 了, 晚上, 的, 夢醒, 來, 後, 還..."
2,"[去, 便利商店, 的, 路上, 看到, 一, 隻, 狗, 它, 對, 我, 搖了搖, 尾巴..."
3,"[今天, 超級, 無敵, 煩躁, 完全, 找, 不到, 原因, 什麼, 都, 不順, 看, ..."
4,"[晚上, 騎腳, 踏車, 時耳機, 播到, 一首, 以前, 常聽, 的, 歌, 突然, 就,..."
5,"[終於, 把手, 邊的, 報告, 交出去, 了, 雖然, 不確, 定做, 得, 好不好, 但..."
6,"[空氣裡, 有, 一種, 快, 下雨, 的, 味道, 這種, 味道, 總讓, 我, 想起, ..."
7,"[想, 說, 的, 話, 太多, 最後還, 是, 決定, 什麼, 都, 不, 說, 靜靜, ..."
8,"[今天, 被, 主管, 誇獎, 了, 一句, 沒什麼, 大不了, 但, 那, 一瞬, 間, ..."
9,"[窗外, 有蟬鳴, 是, 夏天, 要來, 了, 吧, 我, 其實, 不是, 很, 喜歡, 夏..."


In [15]:
word_counts = Counter(all_words)
word_counts

Counter({'今天': 5,
         '圖書館': 1,
         '一整': 1,
         '下午': 1,
         '窗外': 2,
         '陽光剛': 1,
         '好灑': 1,
         '進來照': 1,
         '書頁': 1,
         '整個': 1,
         '像是': 1,
         '靜止': 1,
         '時間裡': 1,
         '也許': 2,
         '就是': 2,
         '一直': 1,
         '安定': 1,
         '有些': 1,
         '出口': 1,
         '變成': 1,
         '晚上': 2,
         '夢醒': 1,
         '還是': 3,
         '一點': 1,
         '情緒': 2,
         '不重': 1,
         '黏著心': 1,
         '便利商店': 1,
         '路上': 1,
         '看到': 1,
         '搖了搖': 1,
         '尾巴': 1,
         '知道': 1,
         '什麼': 3,
         '超級': 1,
         '無敵': 1,
         '煩躁': 1,
         '完全': 1,
         '不到': 1,
         '原因': 1,
         '不順': 1,
         '不爽': 1,
         '討厭': 1,
         '這樣': 2,
         '自己': 3,
         '騎腳': 1,
         '踏車': 1,
         '時耳機': 1,
         '播到': 1,
         '一首': 1,
         '以前': 2,
         '常聽': 1,
         '突然': 2,
         '想起': 2,
         '一起': 1,
 

In [24]:
# 排除停用詞（這裡提供一個簡單的停用詞列表，可根據需要擴展）
stop_words = set(['所以', '好', '因為', '大家', '的', '是', '了', '我', '也', '在', '和', '就', '不', '有', '他', '她', '你', '我們', '這個', '得','今天','開始','什麼','好像','還是','覺得','自己','就是','只是','有點','感覺','有些','其實','這種','真的','可以'])
filtered_word_counts = {word: count for word, count in word_counts.items() if word not in stop_words}

In [25]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df.head(30), x='Word', y='Frequency',
             title='關鍵字出現次數統計（前20名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [26]:
word_freq_df

,Word,Frequency
0,窗外,2
1,也許,2
2,晚上,2
3,情緒,2
4,想起,2
...,...,...
145,不清,1
146,呼吸,1
147,堅強,1
148,習慣,1


In [27]:
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-wqy-zenhei is already the newest version (0.9.45-8).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.


In [28]:
from wordcloud import WordCloud

# 取前 300 個最常出現的詞
top_words = word_freq_df.head(300)

# 使用 WQY Zenhei 字體
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲
wordcloud.generate_from_frequencies(dict(zip(top_words['Word'], top_words['Frequency'])))

# 將文字雲轉換為圖像
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig = px.imshow(wordcloud_img)
fig.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig.show()

In [29]:
from google.colab import userdata
apikey = userdata.get('key')

In [30]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"請摘要出這些留言的重點，並補上重點中的情緒表現，把最多人呈現的情緒及其相關的內容整併在一起，說明這些發文的風向：{df.to_string()}"

response = model.generate_content(prompt)
print(response.text.replace('*',''))

這些留言的風向呈現一種複雜的情緒混合狀態，但以 低落、壓抑與細微的正面情緒 為主。

1. 多數人呈現的情緒：低落與壓抑 (帶有隱藏的情緒)

 內容:  多數留言描述了說不出口的煩惱 (1, 7, 15)，情緒起伏大、感到疲憊與撐不住 (11)，以及對過去美好時光的懷念與現狀的落差 (4, 12)。  即使看似平淡的描述 (6, 13)，也隱含著一種淡淡的愁緒和對生活的無力感。留言 16 更直接點出表面堅強下的脆弱。這些留言共同點是存在著負面情緒，但並未直接爆發，而是以一種壓抑、內斂的方式呈現。

 情緒表現:  低落、壓抑、無力感、淡淡的憂傷、隱藏的脆弱。

2. 少數人呈現的情緒：微小的正面情緒與平靜

 內容:  部分留言表達了在日常生活中找到的微小快樂，例如：在圖書館的寧靜 (0)，看到狗搖尾巴的微笑 (2)，完成報告後的成就感 (5)，茶葉蛋帶來的滿足感 (10)，以及從自然 (風、蟬鳴) 中感受到的活力 (9, 14)。 這些正面情緒比較短暫且微弱，與整體低落的情緒基調形成對比。

 情緒表現:  平靜、滿足感、短暫的快樂、希望。


總結:

這些留言整體呈現出一個內心複雜、情緒壓抑的狀態。雖然部分留言表達了微小的正面情緒，但這些正面情緒並不能掩蓋整體的低落和壓抑感。  發文者們似乎在經歷著一些難以言喻的困境或情緒波動，他們試圖在日常生活中尋找安慰和平衡，但負面情緒仍然佔據了主要部分。  整體風向偏向負面，但並非絕望，而是帶著一種淡淡的哀愁和對未來的期許。



In [32]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"請根據你分析的結果總結出一個代表詞：{df.to_string()}"

response = model.generate_content(prompt)
print(response.text)

代表詞：**情緒波動**

這個月記錄了許多細微的情緒變化，從煩躁、不安到平靜、喜悅，體現了情緒的起伏和不穩定性。  雖然有積極的體驗（例如：在圖書館的安定感、小確幸），但消極情緒（例如：煩躁、壓抑、莫名的失落）也佔據了相當的篇幅。因此，“情緒波動”最能概括這個月的整體狀態。

